In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Convert categorical data to numbers using one-hot encoding
dataset = pd.get_dummies(dataset, columns=['sex', 'smoker', 'region'], drop_first=True)

# Display the first few rows to verify the changes
dataset.head()

In [ ]:
# Split the data into training and testing sets (80% train, 20% test)
# We use random_state for reproducibility
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

# Pop the 'expenses' column to create the labels datasets
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Build the neural network model
model = keras.Sequential([
    # Input layer, with the number of features as the input shape
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    # Hidden layers
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    # Output layer for regression (predicts a single value)
    layers.Dense(1)
])

# Compile the model
# The optimizer adjusts the model's internal weights to minimize loss
# The loss function measures how far off the predictions are from the true values
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='mean_absolute_error',
              metrics=['mae', 'mse'])

# Train the model
# epochs is the number of times the model will cycle through the training data
# verbose=0 silences the output for each epoch
model.fit(train_dataset, train_labels, epochs=200, verbose=0)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
